In [1]:
import tensorflow as tf
import os
os.chdir("..")
os.getcwd()

'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
#model = tf.keras.models.load_model("artifacts/training/model.h5")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories,save_json

In [4]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= 'artifacts/training/model.h5',
            training_data= 'artifacts/data_ingestion/PetImages',
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        
        )

        return eval_config


In [14]:
# training Component
from pathlib import Path
import tensorflow as tf


class Evaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config

    def _valid_generator(self):
        # will be doing augmentation

        datagenerator_kwargs = dict(rescale=1.0 / 255, validation_split=0.30)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[
                :-1
            ],  # only takes rows and columns, channels not aka index 0,1
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            # interpolation is a process of determining the unknown values that lie in between the known data points.
            # ex find value between 1 and 2 on a line.
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # validation data
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path):
        return tf.keras.models.load_model(path)

    # start training
    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0],
                "accuracy":self.score[1]}

        save_json(path=Path("scores.json"),data = scores)
        #store the scores.json in root for dvc to track it easily

In [15]:
try:
    config = ConfigrationManger()
    
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
118/118 [==============================] - 1061s 9s/step - loss: 3.2964 - accuracy: 0.7463
